# Segmenting and Clustering Neighborhoods in Toronto

Set up dependencies

In [115]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import requests # library to fetch URLs

from bs4 import BeautifulSoup # library to scrape data from a webpage

print('Libraries imported.')

Libraries imported.


# Load Data


## Scrape Postal Codes from Wikipedia
Scrape the postal codes table from this Wikipedia page:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

<i>"This is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. Only the first three characters are listed, corresponding to the Forward Sortation Area."</i>


In [136]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response = requests.get(url)
if response.status_code != 200:
    print('Failed to get data:', response.status_code)
else:
    print(f'Data downloaded from: {url}')


Data downloaded from: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [137]:
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find("table", {"class":"wikitable sortable"})
df = pd.read_html(str(table),header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Clean up the postal code dataframe

In [138]:
# Remove unassigned boroughs
n = df.shape[0]
df = df.loc[df['Borough']!='Not assigned',:]
print(f'Removed {n-df.shape[0]:,} rows with unassigned boroughs.')
del n

# Check if a postcode can have more than one borough:
if np.max(df.groupby('Postcode')['Borough'].nunique())==1:
    print('As expected, there is only one borough in each postcode. Yay!')
else:
    print('Watch out! There is at least one postcode with multiple distinct boroughs...')

# Give unassigned neighborhoods the name of their borough
n = sum(df['Neighbourhood']=='Not assigned')
print(f'{n:,} rows with unassigned neighbourhoods (now assigned to borough).')
del n

df.loc[df['Neighbourhood']=='Not assigned','Neighbourhood']=df['Borough']

# Combine multiple rows for a given postcode into one comma-separated list of neighbourhoods
n = df.shape[0]
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
print(f'Collapsed {n-df.shape[0]:,} duplicated postcodes into single rows.')
del n
print('\n')
print(df.head())

Removed 77 rows with unassigned boroughs.
As expected, there is only one borough in each postcode. Yay!
1 rows with unassigned neighbourhoods (now assigned to borough).
Collapsed 108 duplicated postcodes into single rows.


  Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae


In [139]:
print(f'The cleaned dataframe has {df.shape[0]:,} total rows, making up the following buroughs:')
print(df['Borough'].value_counts())

The cleaned dataframe has 103 total rows, making up the following buroughs:
North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64


<b>In the last cell of the notebook, use the .shape method to print the number of rows in the dataframe:</b>

In [140]:
print(df.shape[0])

103


## Get Latitude and Longitude Coordinates

In [141]:
# Download csv from course website
url = 'https://cocl.us/Geospatial_data'
lat_lng_coords = pd.read_csv('https://cocl.us/Geospatial_data')
print(f'Data downloaded from: {url}')

lat_lng_coords.head()

Data downloaded from: https://cocl.us/Geospatial_data


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [142]:
# Combine the two dataframes to create the specified dataframe for this assignment
df = pd.merge(
    left=df, right=lat_lng_coords,
    left_on='Postcode',right_on='Postal Code',
    how='left', validate='1:1', indicator=True)

# Confirm that all of the postal codes matched to a latitude/longitude coordinate
print('Merge results:')
print(df['_merge'].value_counts())

# Adjust the column names to match the assignment
df.drop(['Postal Code','_merge'],axis=1,inplace=True)
df.rename(columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'},inplace=True)

Merge results:
both          103
right_only      0
left_only       0
Name: _merge, dtype: int64


<b>Display the first 15 lines of the dataframe.</b>

In [144]:
df.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
